# Baseball Analytics 

You only need to run the code below the first time

Also need to do this the first time:
 1. Install Firefox is not already installed https://www.mozilla.org/en-US/firefox/new/
 2. Install Geckodriver https://github.com/mozilla/geckodriver/releases
 3. Create a folder `Geckodriver` in your `C:\Program Files` folder on your computer, and put the `geckodriver.exe` file in that folder.

In [1]:
#pip install pandasgui
#pip install selenium
#pip install pickle

In [32]:
#pip install jupyterlab

In [36]:
#pip install ipywidgets

In [46]:
#!jupyter nbextension enable --py --sys-prefix qgrid

In [47]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [41]:
#pip install qgrid

#### Individual Statistics

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import pickle
from IPython.display import display, HTML
import qgrid

In [2]:
teams =["Bethel", "Goshen", "Grace", "HU", "IWU", "Marian", "MVNU", "SAU", "SFU", "Taylor"]
t_nums = [1629, 1678, 1679, 1688, 1694, 1717, 1736, 1780, 1805, 1784]

In [3]:
urls = ['http://www.dakstats.com/WebSync/Pages/Team/IndividualStats.aspx?association=10&sg=MBA&conference=NAIMBA1_CROSS&team=' + str(num) + '&sea=NAIMBA_2019' for num in t_nums]

In [4]:
driver = webdriver.Firefox(executable_path=r'C:\\Program Files\\Geckodriver\\geckodriver.exe')
page_sources = []
for url in urls:
    driver.get(url)
    driver.find_elements_by_css_selector("input[type='radio']")[1].click()
    page_sources.append(driver.page_source)

In [5]:
#Store the page as an element tree using BeautifulSoup4
soups = [BeautifulSoup(page) for page in page_sources]

In [6]:
stat_tables = [[
    [
      [td.get_text(strip=True) for td in tr.find_all('td')] 
      for tr in table.find_all('tr') 
    ]#for each row in each table
    for table in soup.find_all('table',{"class":"gridViewReportBuilderWide"}) 
  ] for soup in soups] #for each table on each webpage

In [7]:
bat_headers = ["Batting", "GP", "GS", "AVG", "AB", "R", "H", "2B", "3B", "HR", "RBI", "TB", "SLG", "BB", "HBP", "SO", "GDP", "OBP", "SF", "SH", "SB", "SBA"]
bat_rows = [stat_tables[i][0] for i in range(len(teams))]

In [8]:
dfb = [pd.DataFrame(columns = bat_headers, data = bat_rows[i]) for i in range(len(teams))]
dfb = [df.iloc[1:] for df in dfb] # remove first empty row

In [9]:
a = qgrid.show_grid(dfb[2], show_toolbar = True)
a

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [10]:
for df in dfb:
    df[["GP", "GS", "AVG", "AB", "R", "H", "2B", "3B", "HR", "RBI", "TB", "SLG", "BB", "HBP", "SO", "GDP", "OBP", "SF", "SH", "SB", "SBA"]] = df[["GP", "GS", "AVG", "AB", "R", "H", "2B", "3B", "HR", "RBI", "TB", "SLG", "BB", "HBP", "SO", "GDP", "OBP", "SF", "SH", "SB", "SBA"]].apply(pd.to_numeric)
dfb[2][:5]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,SO,GDP,OBP,SF,SH,SB,SBA
1,"Griffin, Chris",26,25.0,0.390,82,18,32,4,4,4,18,56,0.683,15,4,21,0,0.505,0,0,0,2
2,"Enyart, Mitchell",23,22.0,0.352,71,11,25,4,0,1,16,32,0.451,11,0,11,1,0.434,1,3,1,1
3,"Harris, Xavier",27,27.0,0.337,86,19,29,3,0,0,14,32,0.372,11,7,16,2,0.448,1,1,3,4
4,"Elford, Sid",18,8.0,0.333,36,5,12,2,0,1,6,17,0.472,4,1,10,2,0.415,0,0,1,1
5,"Haney, Houston",24,23.0,0.321,78,8,25,7,0,2,23,38,0.487,5,2,20,0,0.364,3,0,1,1


In [37]:
pitch_headers = ["Pitching", "ERA", "W", "L", "GP", "GS", "CG", "SHO", "CBO", "SV", "IP", "H", "R", "ER", "BB", "SO", "2B", "3B", "HR", "TBF", "B_AVG", "WP", "HBP", "BK", "SFA", "SHA"]
pitch_rows = [stat_tables[i][1] for i in range(len(teams))]

In [38]:
dfp = [pd.DataFrame(columns = pitch_headers, data = pitch_rows[i]) for i in range(len(teams))]
dfp = [df.iloc[1:] for df in dfp]
dfp[2][:5]

,Pitching,ERA,W,L,GP,GS,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA
1,"Haney, Houston",3.00,2,1,7,4,2,0,0,0,30.0,46,16,10,4,28,10,0,3,140,0.348,0,1,0,2,1
2,"Noska, Jordan",4.71,1,0,10,0,0,0,0,1,21.0,33,15,11,2,15,6,1,2,100,0.367,2,4,1,2,2
3,"Hammel, Jacob",6.00,1,0,5,0,0,0,0,1,6.0,5,4,4,4,9,3,0,0,28,0.217,1,1,0,0,0
4,"Peterson, Ike",7.59,0,0,8,0,0,0,0,1,10.2,15,12,9,1,6,3,0,0,54,0.306,2,3,1,1,0
5,"Anderson, David",8.25,2,3,6,5,1,0,0,0,24.0,43,26,22,14,22,9,4,2,131,0.387,3,3,0,1,2


In [14]:
field_headers = ["Fielding", "GP", "GS", "C", "PO", "A", "E", "FLD_pct", "DP", "TP", "SBA", "RCS", "SB_pct", "PB", "CI", "OBS"]
field_rows = [stat_tables[i][2] for i in range(len(teams))]

In [39]:
dff = [pd.DataFrame(columns = field_headers, data = field_rows[i]) for i in range(len(teams))]
dff = [df.iloc[1:] for df in dff]
dff[2][:5]

,Fielding,GP,GS,C,PO,A,E,FLD_pct,DP,TP,SBA,RCS,SB_pct,PB,CI,OBS
1,"Swartzentruber, Logan",6,5,3,0,3,0,1.000,0,0,0,0,0.000,0,0,0
2,"Clark, Scottie",6,2,1,0,1,0,1.000,0,0,0,0,0.000,0,0,0
3,"Noska, Jordan",10,0,3,0,3,0,1.000,0,0,0,0,0.000,0,0,0
4,"Buzbee, Wyatt",14,0,1,1,0,0,1.000,0,0,0,0,0.000,0,0,0
5,"Haney, Houston",24,23,67,30,36,1,0.985,11,0,0,0,0.000,0,0,0


In [40]:
def calcBatStats(df):
    # Batting Average (useful later for league totals)
    df["AVG"] = round(df["H"] / df["AB"], 3)
    # Caught Stealing
    df["CS"] = round(df["SBA"] - df["SB"], 3)
    # Plate Appearances    
    df["PA"] = round(df["AB"] + df["SF"] + df["SH"] + df["BB"] + df["HBP"])
    # OPS
    df["OPS"] = round(df["OBP"] + df["SLG"], 3)
    # Singles
    df["1B"] = df["H"] - df["2B"] - df["3B"] - df["HR"]
    # Runs Created
    df["RC"] = round((df["H"] + df["BB"]) * df["TB"] / (df["AB"] + df["BB"]), 3)
    # POP
    df["POP"] = round(df["OPS"] + df["AVG"], 3)
    # Total Average
    df["TA"] = round((df["TB"] + df["BB"] + df["HBP"] + df["SB"]) / (df["AB"] - df["H"] + df["SH"] + df["SF"] + df["CS"] + df["GDP"]), 3)
    # Batting Average on Balls in Play
    df["BABIP"] = round((df["H"] - df["HR"]) / (df["AB"] - df["SO"] - df["HR"] + df["SF"]), 3)
    # Hoban Efficiency Quotient - Offense
    df["HEQO"] = df["TB"] + df["R"] + df["RBI"] + df["SB"] + (0.5 * df["BB"])
    # Base Stealing Runs
    df["BSR"] = round(((df["H"] + df["BB"] - df["CS"]) * (df["TB"] + (0.7 * df["SB"]))) / (df["AB"] + df["BB"] + df["CS"]), 3)

In [41]:
for df in dfb:
    calcBatStats(df)
dfb[2][:5]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR
1,"Griffin, Chris",26,25.0,0.390,82,18,32,4,4,4,18,56,0.683,15,4,21,0,0.505,0,0,0,2,2,101,1.188,20,27.134,1.578,1.442,0.491,99.5,25.455
2,"Enyart, Mitchell",23,22.0,0.352,71,11,25,4,0,1,16,32,0.451,11,0,11,1,0.434,1,3,1,1,0,86,0.885,20,14.049,1.237,0.863,0.400,65.5,14.356
3,"Harris, Xavier",27,27.0,0.337,86,19,29,3,0,0,14,32,0.372,11,7,16,2,0.448,1,1,3,4,1,106,0.820,26,13.196,1.157,0.855,0.408,73.5,13.570
4,"Elford, Sid",18,8.0,0.333,36,5,12,2,0,1,6,17,0.472,4,1,10,2,0.415,0,0,1,1,0,41,0.887,9,6.800,1.220,0.885,0.440,31.0,7.080
5,"Haney, Houston",24,23.0,0.321,78,8,25,7,0,2,23,38,0.487,5,2,20,0,0.364,3,0,1,1,0,88,0.851,16,13.735,1.172,0.821,0.390,72.5,13.988


In [42]:
display(dfb[2])

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR
1,"Griffin, Chris",26,25.0,0.390,82,18,32,4,4,4,18,56,0.683,15,4,21,0,0.505,0,0,0,2,2,101,1.188,20,27.134,1.578,1.442,0.491,99.5,25.455
2,"Enyart, Mitchell",23,22.0,0.352,71,11,25,4,0,1,16,32,0.451,11,0,11,1,0.434,1,3,1,1,0,86,0.885,20,14.049,1.237,0.863,0.400,65.5,14.356
3,"Harris, Xavier",27,27.0,0.337,86,19,29,3,0,0,14,32,0.372,11,7,16,2,0.448,1,1,3,4,1,106,0.820,26,13.196,1.157,0.855,0.408,73.5,13.570
4,"Elford, Sid",18,8.0,0.333,36,5,12,2,0,1,6,17,0.472,4,1,10,2,0.415,0,0,1,1,0,41,0.887,9,6.800,1.220,0.885,0.440,31.0,7.080
5,"Haney, Houston",24,23.0,0.321,78,8,25,7,0,2,23,38,0.487,5,2,20,0,0.364,3,0,1,1,0,88,0.851,16,13.735,1.172,0.821,0.390,72.5,13.988
6,"Anderson, David",25,25.0,0.312,109,14,34,6,2,0,13,44,0.404,3,0,11,1,0.327,1,1,2,2,0,114,0.731,26,14.536,1.043,0.628,0.343,74.5,14.998
7,"Brooks, Ethan",17,10.0,0.294,34,7,10,0,1,3,7,21,0.618,2,0,14,0,0.333,0,0,1,2,1,36,0.951,6,7.000,1.245,0.960,0.412,37.0,6.451
8,"Snyder, RJ",25,15.0,0.288,59,21,17,2,0,1,9,22,0.373,3,0,8,2,0.317,1,0,10,14,4,63,0.690,14,7.097,0.978,0.714,0.314,63.5,7.030
9,"Sapp, Jacob",25,23.0,0.277,65,2,18,4,0,1,7,25,0.385,11,2,14,1,0.397,0,0,0,0,0,78,0.782,13,9.539,1.059,0.792,0.340,39.5,9.539
10,"Newkirk, Sam",26,25.0,0.236,89,15,21,2,2,1,9,30,0.337,5,5,22,0,0.307,2,1,2,3,1,102,0.644,16,8.298,0.880,0.583,0.294,58.5,8.263


#### Crossroads League Statistics

In [18]:
bat_totals = [df[df.Batting.str.contains("Total:", regex = False)] for df in dfb]
bat_totals[2]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,...,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR
26,Total:,27,NaN,0.292,872,160,255,38,10,14,...,13,1005,0.777,193,127.076,1.069,0.754,0.359,722.5,127.302


In [19]:
merged_bat_totals = pd.concat(bat_totals)
merged_bat_totals[:4]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,...,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR
22,Total:,27,NaN,0.260,861,115,224,32,3,7,...,7,992,0.667,182,92.841,0.927,0.615,0.330,574.5,97.216
28,Total:,27,NaN,0.254,859,132,218,39,1,15,...,4,1001,0.697,163,101.859,0.951,0.643,0.314,615.5,102.027
26,Total:,27,NaN,0.292,872,160,255,38,10,14,...,13,1005,0.777,193,127.076,1.069,0.754,0.359,722.5,127.302
26,Total:,27,NaN,0.294,843,154,248,60,9,8,...,20,1002,0.798,171,131.480,1.092,0.836,0.364,768.0,139.334


In [20]:
merged_bat_totals.loc["CL_Total"] = merged_bat_totals.sum()
del merged_bat_totals["Batting"]
CL_bat_totals = merged_bat_totals.iloc[-1,:]

In [21]:
CL_tot_b = CL_bat_totals.copy()
# CL On-Base Percentage
CL_tot_b["OBP"] = round((CL_tot_b["H"] + CL_tot_b["BB"] + CL_tot_b["HBP"]) / (CL_tot_b["AB"] + CL_tot_b["BB"] + CL_tot_b["HBP"] + CL_tot_b["SF"]), 3)
# CL Slugging Percentage
CL_tot_b["SLG"] = round(CL_tot_b["TB"] / CL_tot_b["AB"], 3)
# CL weighted On-Base Average (excluding IBB, formula from https://library.fangraphs.com/offense/woba/)
CL_tot_b["wOBA"] = round((0.69*CL_tot_b["BB"] + 0.72*CL_tot_b["HBP"] + 0.89*CL_tot_b["1B"] + 1.27*CL_tot_b["2B"] + 1.62*CL_tot_b["3B"] + 2.10*CL_tot_b["HR"]) / (CL_tot_b["AB"] + CL_tot_b["BB"] + CL_tot_b["SF"] + CL_tot_b["HBP"]), 3)
# CL Runs per Plate Appearance
CL_tot_b["R_per_PA"] = round(CL_tot_b["R"] / CL_tot_b["PA"], 3)
# ABF
CL_tot_b["ABF"] = round((0.47*CL_tot_b["1B"] + 0.38*CL_tot_b["2B"] + 0.55*CL_tot_b["3B"] + 0.93*CL_tot_b["HR"] + 0.33*CL_tot_b["BB"] + CL_tot_b["HBP"]) / (CL_tot_b["AB"] - CL_tot_b["H"]), 3)
# Calculate other totals such as AVG, OPS, POP, etc
calcBatStats(CL_tot_b)
CL_tot_b

GP            270.000
GS              0.000
AVG             0.285
AB           8608.000
R            1548.000
H            2452.000
2B            442.000
3B             53.000
HR            180.000
RBI          1379.000
TB           3540.000
SLG             0.411
BB           1062.000
HBP           241.000
SO           1857.000
GDP           122.000
OBP             0.375
SF             96.000
SH            141.000
SB            339.000
SBA           444.000
CS            105.000
PA          10148.000
OPS             0.786
1B           1777.000
RC           1286.407
POP             1.071
TA              0.783
BABIP           0.341
HEQO         7337.000
BSR          1317.321
wOBA            0.351
R_per_PA        0.153
ABF             0.291
Name: CL_Total, dtype: float64

#### Park Factor

In [22]:
urls = ['http://www.dakstats.com/WebSync/Pages/Team/TeamSchedule.aspx?association=10&sg=MBA&sea=NAIMBA_2019&team=' +
        str(num) for num in t_nums]
#Create a handle, page, to handle the contents of the website
pages = [requests.get(url) for url in urls]
#Store the page as an element tree using BeautifulSoup4
soups = [BeautifulSoup(page.content) for page in pages]

The code below collects all of the html tables from the different teams' webpages on DakStats.

In [23]:
team_tables = [
  [
    [
      [td.get_text(strip=True) for td in tr.find_all('td')] 
      for tr in table.find_all('tr') 
    ]#for each row in each table
    for table in soup.find_all('table') 
  ]#for each table on each webpage
  for soup in soups 
]#for each team's webpage

The below for loop allows us to locate the table from the webpage that contains the data we are interested in. We find the headers in the 33rd table and the actual data in the 35th table. We will assume that this is the same for all teams.

In [24]:
for i in range(len(team_tables[2])):
  #print(i, team_tables[2][i])
  #The line ablve is commented out becuause we only needed to run it once to find the location of the data on the webpage.
  pass

Next, we define the column names for our dataframe.

In [25]:
headers = [['Date', 'Opponent', 'Location', 'Score', 'Outcome'] for tables in team_tables]
headers[2]

['Date', 'Opponent', 'Location', 'Score', 'Outcome']

Here, we collected the data into the list `team_rows`. We used the code `[:5]` to take only the first 5 columns of data and we used the code `[1::2]` to collect the data from every other row, since between each list of data there is an empty list.

In [26]:
team_rows = [[r[:5] for r in tables[35][1::2]] for tables in team_tables]
team_rows[2][:9]

[['2/27/2019', 'Lourdes (Ohio)', 'N', '3-4', 'L'],
 ['2/27/2019', 'Lourdes (Ohio)', 'N', '4-8', 'L'],
 ['3/2/2019', 'Cornerstone (Mich.)', 'N', '3-4', 'L'],
 ['3/2/2019', 'Trinity Baptist', 'N', '5-1', 'W'],
 ['3/4/2019', 'Michigan-Dearborn', 'N', '13-1', 'W'],
 ['3/5/2019', 'Rochester (Mich.)', 'N', '24-4', 'W'],
 ['3/6/2019', 'Robert Morris (Ill.)', 'N', '10-9', 'W'],
 ['3/8/2019', 'Bethel (Ind.) *', 'N', '13-6', 'W'],
 ['3/9/2019', 'Bethel (Ind.) *', 'N', '14-2', 'W']]

Now, we put the data into a datframe.

In [27]:
dfc = [pd.DataFrame(columns = headers[i], data = team_rows[i]) for i in range(len(headers))]
dfc[2][:5]

,Date,Opponent,Location,Score,Outcome
0,2/27/2019,Lourdes (Ohio),N,3-4,L
1,2/27/2019,Lourdes (Ohio),N,4-8,L
2,3/2/2019,Cornerstone (Mich.),N,3-4,L
3,3/2/2019,Trinity Baptist,N,5-1,W
4,3/4/2019,Michigan-Dearborn,N,13-1,W


We subset the data to only include opponents with an asterisk which denotes conference games.

In [28]:
conf_df = [df[df.Opponent.str.contains("*", regex = False)] for df in dfc]
conf_df[2][:5]

,Date,Opponent,Location,Score,Outcome
7,3/8/2019,Bethel (Ind.) *,N,13-6,W
8,3/9/2019,Bethel (Ind.) *,N,14-2,W
9,3/9/2019,Bethel (Ind.) *,N,3-1,W
10,3/14/2019,Taylor (Ind.) *,A,5-15,L
11,3/16/2019,Taylor (Ind.) *,A,2-10,L


The below code copies the dataframe with `.copy()` to avoid errors, splits the "Score" column into two columns, one for the selected team and one for the opponent. Then, the code `str.replace(' \*', '', regex= True)` eliminates the parentheses and the number between them for extra-inning games.

In [29]:
tidy_conf = conf_df.copy()
for i, df in enumerate(conf_df):
  split_scores = df['Score'].str.replace(r"\(.*\)","").str.split('-', expand = True)
  tidy_conf[i] = df.assign(Score = pd.to_numeric(split_scores[0]),
                           Opp_score = pd.to_numeric(split_scores[1]),
                           Opponent = df.Opponent.str.replace(' \*', '', regex= True),
                           Date = pd.to_datetime(df.Date)
                           )
tidy_conf[2][:5]

,Date,Opponent,Location,Score,Outcome,Opp_score
7,2019-03-08,Bethel (Ind.),N,13,W,6
8,2019-03-09,Bethel (Ind.),N,14,W,2
9,2019-03-09,Bethel (Ind.),N,3,W,1
10,2019-03-14,Taylor (Ind.),A,5,L,15
11,2019-03-16,Taylor (Ind.),A,2,L,10


Finally, we subset the home games in one dataframe and the away games in another, and then use the sum of the two score columns in each to calculate our park factor for each team.

In [30]:
conf_h = [df[df.Location.str.contains("H", regex = False)] for df in tidy_conf]
conf_a = [df[df.Location.str.contains("A", regex = False)] for df in tidy_conf]

In [31]:
h_runs_per_game = [(df.Score.sum() + df.Opp_score.sum())/len(df.index) for df in conf_h]
a_runs_per_game = [(df.Score.sum() + df.Opp_score.sum())/len(df.index) for df in conf_a]
park_factor = [h_runs_per_game[i]/a_runs_per_game[i] for i in range(len(headers))]
park_factor

[0.963768115942029,
 1.1224268689057422,
 0.8870967741935484,
 0.7320261437908496,
 1.0357675111773472,
 0.7938931297709924,
 0.8571428571428571,
 1.15,
 1.3483365949119375,
 1.3363844393592679]